In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd drive/MyDrive/Capstone\ 3

/content/drive/MyDrive/Capstone 3


In [4]:
import pandas as pd
import os 
import ast
import matplotlib.pyplot as plt 
from sklearn.metrics.pairwise import cosine_similarity ### Cosine Similary
from scipy import sparse ### Sparse Matrix
import numpy as np

Loaded in original interation dataframe and recippe dataframe and only kept the recipes that had more than 2 reviews

In [5]:
interactions = pd.read_csv('data/RAW_interactions.csv')
recipes = pd.read_csv('data/RAW_recipes.csv')

In [12]:
g = {'rating' : ['mean'],'user_id' : ['nunique']}
int_summary = interactions.groupby(['recipe_id']).agg(g).reset_index()

In [13]:
#found average of every recipe and counted the number of unique reviewrs
int_summary

,recipe_id,rating,user_id
,,mean,nunique
0,38,4.250000,4
1,39,3.000000,1
2,40,4.333333,9
3,41,4.500000,2
4,43,1.000000,1
...,...,...,...
231632,537459,5.000000,1
231633,537485,5.000000,1
231634,537543,0.000000,1


In [14]:
#rename the columns 
ind = pd.Index([e[0] + "_" +e[1] for e in int_summary.columns.tolist()])
int_summary.columns = ind
int_summary.columns = ['recipe_id', 'rating_mean', 'user_id_nunique']
int_summary

,recipe_id,rating_mean,user_id_nunique
0,38,4.250000,4
1,39,3.000000,1
2,40,4.333333,9
3,41,4.500000,2
4,43,1.000000,1
...,...,...,...
231632,537459,5.000000,1
231633,537485,5.000000,1
231634,537543,0.000000,1
231635,537671,0.000000,1


In [15]:
#only keep the recipes with more than 2 unique users
int_summary_94k = int_summary[ (int_summary['user_id_nunique'] > 2)]

In [17]:
#Merge the recipe data with the filtered recipes with more than 2 unique reviewers
filter_recipe = pd.merge(recipes,int_summary_94k,right_on = ['recipe_id'],left_on = ['id'],how = 'inner')

In [ ]:
#How to extract the nutriton values from the nutrition column in dataframe
test_df = filter_recipe.copy()
test_df['calories'] = test_df['nutrition'].map(lambda x: ast.literal_eval(x)[0])
test_df['total fat'] = test_df['nutrition'].map(lambda x: ast.literal_eval(x)[1])
test_df['sugar'] = test_df['nutrition'].map(lambda x: ast.literal_eval(x)[2])
test_df['sodium'] = test_df['nutrition'].map(lambda x: ast.literal_eval(x)[3])
test_df['protien'] = test_df['nutrition'].map(lambda x: ast.literal_eval(x)[4])
test_df['saturated fat'] = test_df['nutrition'].map(lambda x: ast.literal_eval(x)[5])
test_df['carbohydrates'] = test_df['nutrition'].map(lambda x: ast.literal_eval(x)[6])

In [ ]:
# To turn the recipe into one sentence for tokenization use ast.literal_eval()
test_df['dish_recipe'] = filter_recipe['steps'].apply(lambda x : " ".join(ast.literal_eval(x)))

- In AWS I was able to use their pytorch GPU kernal for the computing of the embedding in BERT and cosine similarity of the first 10,000 recipes.

- The way the recipes are compared to one another was by the way the dish was made. In the steps of how to make the dish not only includeded ingredients but also gave bert the chance to understand the meaning of words and phrases. 

- After running the cosine similarity I saved only the top 10 closest recipes for each individual recipe.

- I than saved the 90k recipes and cosine df into csv to contine exploration in google colab

In [ ]:
recipes = pd.read_csv('data/90_k_recipe.csv')

In [ ]:
recipes = recipes[:10000]

In [ ]:
top_10_for_each_recipe = pd.read_csv('data/first_10k_top_10_rec.csv')

In [ ]:
top_10_for_each_recipe.head(11)

,Unnamed: 0,recipe1,recipe2,cosine_similarity,recipe1_name,recipe2_name,similarity_rank
0,0,0,1087,0.824837,arriba baked winter squash mexican style,acorn squash a la kree,3.0
1,1,0,1090,0.824706,arriba baked winter squash mexican style,acorn squash bisque,4.0
2,2,0,1094,0.807394,arriba baked winter squash mexican style,acorn squash roasted with applesauce,7.0
3,3,0,1101,0.817159,arriba baked winter squash mexican style,acorn squash with pineapple,5.0
4,4,0,1102,0.813644,arriba baked winter squash mexican style,acorn squash with raisin sauce,6.0
5,5,0,4184,0.797381,arriba baked winter squash mexican style,aunt fanny s baked yellow squash,8.0
6,6,0,4436,0.793161,arriba baked winter squash mexican style,autumn baked acorn squash,10.0
7,7,0,5816,0.833334,arriba baked winter squash mexican style,baked delicata squash with lime butter,2.0
8,8,0,6352,0.835143,arriba baked winter squash mexican style,baked spaghetti squash with jalapeno cream,1.0
9,9,0,7628,0.796396,arriba baked winter squash mexican style,barefoot contessa s butternut squash risotto,9.0


In [ ]:
recipes.drop('nutrition',axis=1,inplace=True)

In [ ]:
recipes_features = recipes[['name','n_steps','n_ingredients','rating_mean','calories','total fat','sugar','sodium','protien','saturated fat','carbohydrates']]

In [ ]:
recipes_features.columns

Index(['name', 'n_steps', 'n_ingredients', 'rating_mean', 'calories',
       'total fat', 'sugar', 'sodium', 'protien', 'saturated fat',
       'carbohydrates'],
      dtype='object')

In [ ]:
feat = ['calories',
       'total fat', 'sugar', 'sodium', 'protien', 'saturated fat',
       'carbohydrates']

In [ ]:
X = recipes_features[feat].values

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
ss = StandardScaler()

In [ ]:
X = ss.fit_transform(X)

- Along with the embedding of the recipes the nutrition values extracted in the begining can be used to recommend.

In [ ]:
#Cosine similarity with the nutrition features

def cos_sim(name, X, n=5):
    index = recipes.index[(recipes['name'] == name)][0]
    rec = X[index].reshape(1,-1)
    cs = cosine_similarity(rec, X)
    rec_index = np.argsort(cs)[0][::-1]
    ordered_df = recipes.loc[rec_index]
    ordered_df = ordered_df.drop(index)
    rec_df = ordered_df.head(n)
    orig_row = recipes.loc[[index]].rename(lambda x: 'original')
    total = pd.concat((orig_row,rec_df))
    total.drop('Unnamed: 0',axis=1,inplace=True)
    return total

In [ ]:
cos_sim('arriba   baked winter squash mexican style',X)

,Unnamed: 0,name,id,minutes,contributor_id,submitted,tags,n_steps,steps,description,ingredients,n_ingredients,recipe_id,rating_mean,user_id_nunique,calories,total fat,sugar,sodium,protien,saturated fat,carbohydrates,dish_recipe
original,0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,137739,5.000000,3,51.5,0.0,13.0,0.0,2.0,0.0,4.0,make a choice and proceed with recipe dependin...
6418,6418,baked sweet potato sticks,4765,15,1533,1999-11-21,"[15-minutes-or-less, time-to-make, course, mai...",7,"['preheat oven to 450f', 'lightly oil a baking...",NaN,"['sweet potatoes', 'salt & pepper']",2,4765,4.500000,14,55.9,0.0,10.0,1.0,2.0,0.0,4.0,preheat oven to 450f lightly oil a baking shee...
2122,2122,amish turnips,60720,75,58407,2003-04-22,"[weeknight, time-to-make, course, main-ingredi...",7,"['cook turnips until tender', 'drain , mash an...",very good recipe! i never had mashed turnips b...,"['turnips', 'breadcrumbs', 'oleo', 'brown suga...",7,60720,4.666667,18,158.7,9.0,33.0,8.0,10.0,10.0,7.0,"cook turnips until tender drain , mash and the..."
8074,8074,batter fried onion rings,33676,50,37305,2002-07-09,"[60-minutes-or-less, time-to-make, course, mai...",9,['cover onion slices with cold water and let s...,these are great homemade onion rings that are ...,"['flour', 'salt', 'onions', 'milk', 'sugar', '...",8,33676,3.454545,11,92.8,3.0,14.0,4.0,5.0,3.0,5.0,cover onion slices with cold water and let sta...
635,635,35 calorie pumpkin cookies,333395,32,1003473,2008-10-28,"[60-minutes-or-less, time-to-make, course, pre...",9,"['preheat oven to 350 degrees f', 'combine the...",these cookies are absolutely delicious and pra...,"['canned pumpkin', 'egg', 'plain yogurt', 'hon...",13,333395,2.750000,4,61.1,0.0,13.0,4.0,3.0,1.0,4.0,preheat oven to 350 degrees f combine the firs...
1101,1101,acorn squash with pineapple,28507,65,17803,2002-05-15,"[weeknight, time-to-make, course, main-ingredi...",10,"['preheat oven to 350 degrees', 'cut squash in...",doubles nicely.,"['acorn squash', 'cinnamon', 'nutmeg', 'allspi...",6,28507,2.600000,5,60.7,0.0,10.0,0.0,2.0,0.0,5.0,preheat oven to 350 degrees cut squash in half...


In [ ]:
recipes_ingredients = recipes[['name','ingredients']]

In [ ]:
recipes_ingredients['ingredients'] = recipes_ingredients['ingredients'].map(lambda x: ast.literal_eval(x)) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
ingredients = ['potato']

def recipe_with_ingredients(ingredients,n):
  r_cop = recipes.copy()
  for i, ig in enumerate(recipes_ingredients['ingredients']):
    count = len(ingredients)
    for word in ingredients:
      if word in ig:
        count -= 1
    if count != 0:
      r_cop.drop(i,inplace=True)
  r_cop.drop('Unnamed: 0',axis=1,inplace=True)
  return r_cop.sort_values('rating_mean',ascending=False)[:n]


In [ ]:
recipe_with_ingredients(ingredients,10)

,name,id,minutes,contributor_id,submitted,tags,n_steps,steps,description,ingredients,n_ingredients,recipe_id,rating_mean,user_id_nunique,calories,total fat,sugar,sodium,protien,saturated fat,carbohydrates,dish_recipe
5931,baked grated beets,87214,40,133962,2004-03-21,"[60-minutes-or-less, time-to-make, course, mai...",8,"['peel beets and shred coarsely', 'peel and gr...",a tasty sweet and sour dish that is extremely ...,"['beets', 'onion', 'potato', 'vegetable oil', ...",9,87214,5.0,3,125.4,5.0,51.0,14.0,4.0,2.0,7.0,peel beets and shred coarsely peel and grate o...
7677,barefoot contessa s vegetable tian ina garten,196404,85,919937,2006-11-19,"[time-to-make, course, main-ingredient, prepar...",11,"['preheat the oven to 375 degrees f', 'brush a...",we had this for thanksgiving 2005 . it was won...,"['olive oil', 'yellow onions', 'garlic cloves'...",11,196404,5.0,6,109.8,4.0,20.0,10.0,9.0,7.0,5.0,preheat the oven to 375 degrees f brush a 9 by...
4897,baby food chicken,75911,120,50441,2003-11-07,"[weeknight, time-to-make, preparation, for-1-o...",6,"['grill or bake chicken , remove bones and shr...",i used to make my own baby food when the kids ...,"['butter', 'unbleached flour', 'milk', 'chicke...",8,75911,5.0,3,219.8,19.0,3.0,5.0,12.0,39.0,7.0,"grill or bake chicken , remove bones and shred..."
610,30 minute baked potato,155496,25,252156,2006-02-10,"[lactose, 30-minutes-or-less, time-to-make, co...",6,['stab the potato with a fork several times an...,"cut the time, but not the lovely oven cooked t...","['potato', 'olive oil']",2,155496,5.0,12,164.0,0.0,6.0,0.0,8.0,0.0,12.0,stab the potato with a fork several times and ...
8284,bc ferries manhattan clam chowder,238076,65,281339,2007-07-01,"[time-to-make, course, main-ingredient, cuisin...",6,"['in a large saucepan , over medium heat , sau...",this is from a free recipe card that they used...,"['bacon', 'celery', 'diced onion', 'green pepp...",14,238076,5.0,4,185.8,11.0,25.0,21.0,18.0,11.0,6.0,"in a large saucepan , over medium heat , saute..."
2054,amish bean soup,358385,240,465829,2009-03-01,"[ham, time-to-make, course, main-ingredient, c...",5,['boil the beans in the water for 2 minutes be...,this is a wonderful warm soup to warm the hear...,"['dried navy beans', 'water', 'ham bone', 'gar...",10,358385,5.0,4,274.8,1.0,20.0,1.0,33.0,0.0,17.0,boil the beans in the water for 2 minutes befo...
1118,adas polo persian rice with lentils,430878,70,973916,2010-06-24,"[time-to-make, cuisine, preparation, healthy, ...",25,"['in a large saucepan , bring salted water to ...",a delicious and hearty iranian dish made of ar...,"['lentils', 'basmati rice', 'water', 'onion', ...",18,430878,5.0,6,747.5,22.0,149.0,29.0,34.0,39.0,47.0,"in a large saucepan , bring salted water to a ..."
611,30 minute beef pot pie for 2,71698,30,56112,2003-09-26,"[30-minutes-or-less, time-to-make, course, mai...",10,['bake biscuits according to package direction...,this recipe came about because of a foodtv,"['stew meat', 'chicken broth', 'olive oil', 'm...",10,71698,4.9,10,1143.9,92.0,34.0,75.0,81.0,88.0,36.0,bake biscuits according to package directions ...
1392,algerian poached eggs chakchouka,233490,30,169969,2007-06-08,"[30-minutes-or-less, time-to-make, course, mai...",13,"['in a deep skillet , heat the oil over medium...",these eggs are poached in a delicious in a pep...,"['olive oil', 'cumin seed', 'paprika', 'onion'...",19,233490,4.8,5,252.8,23.0,25.0,3.0,18.0,15.0,6.0,"in a deep skillet , heat the oil over medium h..."
8641,beef minestrone,411048,80,383346,2010-01-31,"[time-to-make, course, main-ingredient, prepar...",5,"['in a dutch oven , cook beef and onion over m...",a recipe from taste of home. it's low-salt and...,"['lean ground beef', 'onion', 'water', 'potato...",13,411048,4.8,5,129.7,7.0,8.0,2.0,21.0,10.0,3.0,"in a dutch oven , cook beef and onion over med..."


In [ ]:
for number, step in enumerate(r_cop.sort_values('rating_mean',ascending=False)[:5]['steps'].map(lambda x: ast.literal_eval(x)).iloc[0]):
  print(f'Step{ number} : {step}')


Step0 : heat oven to 350
Step1 : grease& flour two 9x5 loaf pans
Step2 : combine flour , salt& soda- sift together in medium bowl , and set aside
Step3 : in large mixing bowl , cream sugar and shortening
Step4 : add eggs , sour cream& orange juice
Step5 : stir flour in until just moistened
Step6 : fold in cranberries , shredded apple and nuts
Step7 : divide mixture between prepared loaf pans
Step8 : bake 50 minutes , or until a tester comes out clean
Step9 : bread should be golden brown
Step10 : cool 10 minutes in pans , then cool on rack until cool
Step11 : wrap each loaf tightly and store in refrigerator for 2 days before slicing , or freeze


In [ ]:
top_10_for_each_recipe.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
top_10_for_each_recipe[top_10_for_each_recipe['recipe1_name'] == 'amazing chili'].sort_values('similarity_rank')

,recipe1,recipe2,cosine_similarity,recipe1_name,recipe2_name,similarity_rank
19055,1916,8504,0.915293,amazing chili,beef and onion pie,1.0
19050,1916,3466,0.901533,amazing chili,armenian dinner sandwich,2.0
19053,1916,6748,0.900648,amazing chili,baltimore hash,3.0
19058,1916,8897,0.899948,amazing chili,beefy refried bean soup,4.0
19052,1916,6670,0.899578,amazing chili,balsamic pot roast crock pot,5.0
19049,1916,732,0.898039,amazing chili,5 bean chili,6.0
19056,1916,8641,0.896802,amazing chili,beef minestrone,7.0
19057,1916,8833,0.896319,amazing chili,beef with cabbage and tomatoes,8.0
19051,1916,6164,0.894292,amazing chili,baked penne with ground beef and tomato sauce,9.0
19054,1916,8458,0.892335,amazing chili,beef and barley chili,10.0
